## Application des transformations definies dans "trasnfromers.py"

In [2]:
!pip install alembic==1.14.0 \
catboost==1.2.7 \
colorlog==6.9.0 \
contourpy==1.3.1 \
cycler==0.12.1 \
fonttools==4.55.0 \
graphviz==0.20.3 \
greenlet==3.1.1 \
joblib==1.4.2 \
kiwisolver==1.4.7 \
Mako==1.3.6 \
MarkupSafe==3.0.2 \
matplotlib==3.9.3 \
numpy==1.26.4 \
nvidia-nccl-cu12==2.23.4 \
optuna==4.1.0 \
packaging==24.2 \
pandas==2.2.3 \
pillow==11.0.0 \
plotly==5.24.1 \
pyparsing==3.2.0 \
python-dateutil==2.9.0.post0 \
pytz==2024.2 \
PyYAML==6.0.2 \
scikit-learn==1.5.2 \
scipy==1.14.1 \
six==1.16.0 \
SQLAlchemy==2.0.36 \
tenacity==9.0.0 \
threadpoolctl==3.5.0 \
tqdm==4.67.1 \
typing_extensions==4.12.2 \
tzdata==2024.2 \
xgboost==2.1.3


  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 4.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00ta 0:00:01
  Using cached optuna-4.1.0-py3-none-any.whl.metadata (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 304.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 140.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 34.4 MB/s eta 0:00:00:00:0100:01
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 3.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 57.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np
import optuna
import catboost as cb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
from transformers import *

from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')
columns_to_drop = [
    "piezo_station_department_name",
    "piezo_station_update_date",
    "piezo_station_commune_code_insee",
    "piezo_station_pe_label",
    "piezo_station_bdlisa_codes",
    "piezo_station_bss_code",
    "piezo_station_bss_id",
    "piezo_bss_code",
    "piezo_measurement_date",
    "piezo_producer_name",
    "piezo_measure_nature_code",
    "meteo_name",
    "meteo_id",
    "meteo_latitude",
    "meteo_longitude",
    "hydro_station_code",
    "hydro_method_code",
    "hydro_method_label",
    "hydro_qualification_label",
    "prelev_structure_code_0",
    "prelev_structure_code_2",
    "prelev_structure_code_0",
    "prelev_commune_code_insee_0",
    "piezo_station_department_code",

    "meteo_DRR",
    "meteo_temperature_min_ground",
    "meteo_temperature_min_50cm",
    "meteo_pressure_avg",
    "meteo_pression_maxi",
    "meteo_wind_speed_avg_2m",
    "meteo_wind_max_2m",
    "meteo_wind_direction_max_inst_2m",
    "meteo_time_wind_max_2m",
    "meteo_wetting_duration",
    "meteo_sunshine_duration",
    "meteo_radiation",
    "meteo_radiation_direct",
    "meteo_sunshine_%",
    "meteo_radiation_IR",
    "meteo_radiation_UV_max",
    "meteo_cloudiness",
    "meteo_cloudiness_height",
    "meteo_if_snow",
    "meteo_if_fog",
    "meteo_if_thunderstorm",
    "meteo_if_sleet",
    "meteo_if_hail",
    "meteo_if_dew",
    "meteo_if_black_ice",
    "meteo_if_snow_ground",
    "meteo_if_frost",
    "meteo_if_smoke",
    "meteo_if_mist",
    "meteo_if_lightning",
    "meteo_evapotranspiration_Monteith",
    "meteo_radiation_UV",
    "meteo_snow_height",
    "meteo_snow_thickness_max",
    "meteo_snow_thickness_6h"]


# columns_to_drop += ajouts_drop_yael
path_src_dataset = Path("/home/jovyan/hfactory_magic_folders/water_shortage_prediction/X_train_Hi5.csv")

col_yass = ['meteo_date'] + ['prelev_volume_0', 'prelev_volume_1',
                             'prelev_volume_2', 'prelev_other_volume_sum', 'piezo_station_commune_name']
# Altitude
altitude_flo = ["piezo_station_altitude", "meteo_altitude"]  # ORDRE 1
prelev_flo = ["prelev_volume_0", "prelev_usage_label_0", "prelev_volume_obtention_mode_label_0", "prelev_volume_1", "prelev_usage_label_1",
              "prelev_volume_obtention_mode_label_1", "prelev_volume_2", "prelev_usage_label_2", "prelev_volume_obtention_mode_label_2"]
col_flo = altitude_flo + prelev_flo
# Insee & rain "CleanFeatures"
cols_yael_input = ['insee_%_agri', 'meteo_rain_height', 'insee_pop_commune',
                   'insee_med_living_level', 'insee_%_ind', 'insee_%_const']
cols_yael_need = ["piezo_station_department_code", "meteo_date"]

# Temperature
cols_lucien_need = ['piezo_station_department_code', 'piezo_measurement_date']
cols_lucien_input = ['meteo_temperature_avg', 'meteo_temperature_min',
                     'meteo__pressure_saturation_avg', 'meteo_temperature_max']
# Lat Long
cols_mat = ["distance_piezo_meteo", 'piezo_station_longitude', 'piezo_station_latitude', 'meteo_latitude', 'meteo_longitude', "meteo_temperature_avg", "meteo_temperature_avg_threshold",
            "meteo_temperature_min", "meteo_temperature_max", "meteo_temperature_min_ground", "hydro_observation_result_elab", "hydro_status_code", "hydro_qualification_code", "hydro_hydro_quantity_elab"]


# Clean pizo
pizo_cols = ['piezo_station_investigation_depth', 'piezo_obtention_mode', 'piezo_status',
             'piezo_qualification', 'piezo_measure_nature_code', 'piezo_station_department_code']

# target
target = "piezo_groundwater_level_category"


columns_to_keep = col_yass + cols_yael_input + cols_yael_need + col_flo + \
    cols_lucien_need + cols_lucien_input + cols_mat + [target] + pizo_cols

# Out folders
out_folder_dataset = Path("./data/cleaned")
# Create the folder if it doesn't exist
out_folder_dataset.mkdir(parents=True, exist_ok=True)
out_folder_config = Path("./data/processed/pipelines")
out_folder_config.mkdir(parents=True, exist_ok=True)

# Load the CSV file with only the relevant columns
# SI on veut charger moins de lignes : ajouter --> ,nrows=10e4)
df = pd.read_csv(path_src_dataset, usecols=columns_to_keep)
df = df.drop_duplicates()
categorical_var = ['distance_piezo_meteo',
                   'prelev_usage_label_0_EAU POTABLE',
                   'prelev_usage_label_0_EAU TURBINEE (barrage)',
                   'prelev_usage_label_0_nan',
                   'prelev_usage_label_0_infrequent_sklearn',
                   'prelev_volume_obtention_mode_label_0_Mesure directe',
                   'prelev_volume_obtention_mode_label_0_Volume mesuré',
                   'prelev_volume_obtention_mode_label_0_nan',
                   'prelev_volume_obtention_mode_label_0_infrequent_sklearn']

cont_cols = ['piezo_station_investigation_depth',
             'piezo_station_altitude',
             'piezo_station_longitude',
             'piezo_station_latitude',
             'meteo_date',
             'meteo_rain_height',
             'meteo_temperature_avg',
             'meteo__pressure_saturation_avg',
             'hydro_observation_result_elab',
             'hydro_status_code',
             'hydro_qualification_code',
             'prelev_volume_0',
             'prelev_other_volume_sum',
             'insee_%_agri',
             'insee_pop_commune',
             'insee_med_living_level',
             'insee_%_ind',
             'insee_%_const',
             'hydro_observation_log',
             'hydro_hydro_quantity_elab_infrequent_sklearn',
             "piezo_obtention_mode_Mode d'obtention inconnu",
             'piezo_obtention_mode_Valeur mesurée',
             'piezo_obtention_mode_Valeur reconstituée',
             'piezo_status_Donnée brute',
             'piezo_status_Donnée contrôlée niveau 1',
             'piezo_status_Donnée contrôlée niveau 2',
             'piezo_status_Donnée interprétée',
             'piezo_qualification_Correcte',
             'piezo_qualification_Incertaine',
             'piezo_qualification_Incorrecte',
             'piezo_qualification_Non qualifié',
             'piezo_measure_nature_code_0',
             'piezo_measure_nature_code_D',
             'piezo_measure_nature_code_I',
             'piezo_measure_nature_code_N',
             'piezo_measure_nature_code_S']



df['meteo_date'] = pd.to_datetime(df['meteo_date'], errors='coerce')
df = df[(df['meteo_date'].dt.month >= 6) & (df['meteo_date'].dt.month <= 10)]

X = df.drop(columns=target)


# Mapping du target
mapping = {'Very Low': 0, 'Low': 1, 'Average': 2, 'High': 3, 'Very High': 4}
y = df[target].map(mapping)


# Test-val split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42)


# Apply the transformers selected
processing_pipeline = Pipeline(steps=[
    ("DropNaRate", DropNaRate(0.7)),
    ('PrelevVol', PrelevVol()),
    ("Prelevement", Prelev(columns=col_flo, usage_label_max_categories=4,
     mode_label_max_categories=4, scale=3)),
    ("CleanFeatures", CleanFeatures(cols_yael_input)),
    ("Altitude", AltitudeTrans(columns=[
     "piezo_station_altitude", "meteo_altitude"])),
    ('LatLong', CleanLatLon()),
    ('CleanTemp', CleanTemp()),
    ('Temp', TemperaturePressionTrans(columns=cols_lucien_input)),
    ('CleanHydro', CleanHydro()),
    ('CleanPizo',  CleanPizo(pizo_cols)),
    ('Dates', DateTransformer()),
    ('DropCols', DropCols(columns_to_drop)),
    ('scaler', PartialStandardScaler(columns=cont_cols)),
])

In [5]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler  # Use if PartialStandardScaler is unavailable

global_pipeline = Pipeline([
    ('processing', processing_pipeline),  # Assuming processing_pipeline is defined elsewhere  
    ('estimator', XGBClassifier(
        n_estimators=369,
        learning_rate=0.0799316798896716,
        max_depth=15
    ))
])

In [6]:
global_pipeline.fit(X_train, y_train)

>> (Info) Droped columns : []
>> (INFO) missing values in columns ['prelev_volume_0', 'prelev_volume_1', 'prelev_volume_2', 'prelev_other_volume_sum'] are filled by the minimum of the column by commune
>> (Info - Prelev) 'prelev_usage_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_usage_label_1' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_1' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_usage_label_2' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_2' has been one-hot-encoded in 4 features
>> (Info) Calculating medians and means for ['insee_%_agri', 'meteo_rain_height', 'insee_pop_commune', 'insee_med_living_level', 'insee_%_ind', 'insee_%_const']
>> (Info) Rainfall means by department and month calculated.
>> (Info) Medians and means successfully c

Pipeline(steps=[('processing',
                 Pipeline(steps=[('DropNaRate', DropNaRate(rate=0.7)),
                                 ('PrelevVol', PrelevVol()),
                                 ('Prelevement',
                                  Prelev(columns=['piezo_station_altitude',
                                                  'meteo_altitude',
                                                  'prelev_volume_0',
                                                  'prelev_usage_label_0',
                                                  'prelev_volume_obtention_mode_label_0',
                                                  'prelev_volume_1',
                                                  'prelev_usage_label_1',
                                                  'prelev_volume_obtention_mode_label_1',
                                                  'prelev_v...
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.0799316798896716, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=15,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=369,
                               n_jobs=None, num_parallel_tree=None,
                               objective='multi:softprob', ...))])

In [8]:
from sklearn.metrics import classification_report, accuracy_score

In [9]:
y_val_pred = global_pipeline.predict(X_val)

# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))

>> (Info - Prelev) 'prelev_usage_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_0' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_usage_label_1' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_1' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_usage_label_2' has been one-hot-encoded in 4 features
>> (Info - Prelev) 'prelev_volume_obtention_mode_label_2' has been one-hot-encoded in 4 features
>> (Info) Filling missing values with calculated medians and means.
>> (Info) Transforming data: Filling missing values and applying one-hot encoding.
>> (Info) Missing values in piezo_station_investigation_depth filled with department means or global mean as fallback.
>> (Info) One-hot encoding applied to piezo_obtention_mode with missing values filled.
>> (Info) One-hot encoding applied to piezo_status with missing values filled.
>> (Info) One-hot encoding app

### Save Pipeline

In [11]:

# Out folders
out_folder_dataset = Path("./data/cleaned")
out_folder_dataset.mkdir(parents=True, exist_ok=True) # Create the folder if it doesn't exist
out_folder_config = Path("./data/processed/pipelines")
out_folder_config.mkdir(parents=True, exist_ok=True)

In [10]:
save = True
if save:
        
    pipeline_name = "pip_yael_1h45"

    # Writing to sample.json
    with open(out_folder_config / Path(pipeline_name + ".pkl"), "wb") as file:
        pickle.dump(global_pipeline, file)